# Getting started with prompty


**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using prompty and visualize the trace of your application.
- batch run prompty against multi lines of data.


## 0. Install dependent packages

In [1]:
%%capture --no-stderr
%pip install promptflow-core

## 1. Execute a Prompty

Prompty is a file with .prompty extension for developing prompt template. 
The prompty asset is a markdown file with a modified front matter. 
The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [2]:
with open("basic.prompty") as fin:
    print(fin.read())

---
name: Basic Prompt
description: A basic prompt that uses the chat API to answer questions
model:
    api: chat
    configuration:
        type: azure_openai
        azure_deployment: gpt-4
    parameters:
        max_tokens: 128
        temperature: 0.2
inputs:
  question:
    type: string
sample:
  "question": "Who is the most famous person in the world?"
---
system:
You are an AI assistant who helps people find information.
As the assistant, you answer questions briefly, succinctly. 

user:
{{question}}


Note: before running below cell, please configure required environment variable `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT` by create an `.env` file. Please refer to `../.env.example` as an template.


In [3]:
import os
from dotenv import load_dotenv

if "AZURE_OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv()

In [4]:
"AZURE_OPENAI_API_KEY" not in os.environ

False

In [5]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load(source="basic.prompty")

# execute the flow as function
result = f(question="What is the capital of France?")
result

'The capital of France is Paris.'

You can override configuration with `AzureOpenAIModelConfiguration` and `OpenAIModelConfiguration`.

In [6]:
from promptflow.core import AzureOpenAIModelConfiguration, OpenAIModelConfiguration

# override configuration with AzureOpenAIModelConfiguration
configuration = AzureOpenAIModelConfiguration(
    # azure_endpoint="${env:AZURE_OPENAI_ENDPOINT}",  # Use ${env:<ENV_NAME>} to surround the environment variable name.
    # api_key="${env:AZURE_OPENAI_API_KEY}",
    azure_deployment="gpt-4o",
)

# override configuration with OpenAIModelConfiguration
# configuration = OpenAIModelConfiguration(
#     base_url="${env:OPENAI_BASE_URL}",
#     api_key="${env:OPENAI_API_KEY}",
#     model="gpt-3.5-turbo"
# )

override_model = {"configuration": configuration, "parameters": {"max_tokens": 512}}

# load prompty as a flow
f = Prompty.load(source="basic.prompty", model=override_model)

# execute the flow as function
result = f(question="What is the capital of France?")
result

'The capital of France is Paris.'

### Visualize trace by using start_trace

In [7]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Prompt flow service has started...


Re-run below cell will collect a trace in trace UI.

In [8]:
# rerun the function, which will be recorded in the trace
question = "What is the capital of Japan?"
ground_truth = "Tokyo"
result = f(question=question)
result

'The capital of Japan is Tokyo.'

### Eval the result 

Note: the eval flow returns a `json_object`.

In [9]:
# load prompty as a flow
eval_flow = Prompty.load("../eval-basic/eval.prompty")
# execute the flow as function
result = eval_flow(question=question, ground_truth=ground_truth, answer=result)
result

{'score': '5', 'explanation': 'Tokyo is the capital of Japan.'}

## 2. Batch run with multi-line data


In [10]:
%%capture --no-stderr
# batch run requires promptflow-devkit package
%pip install promptflow-devkit

In [11]:
from promptflow.client import PFClient

pf = PFClient()

In [12]:
flow = "./basic.prompty"  # path to the prompty file
data = "./data.jsonl"  # path to the data file

# create run with the flow and data
base_run = pf.run(
    flow=flow,
    data=data,
    column_mapping={
        "question": "${data.question}",
    },
    stream=True,
)

[2025-02-27 14:44:14 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run basic_20250227_144414_802786, log path: /home/azureuser/.promptflow/.runs/basic_20250227_144414_802786/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=basic_20250227_144414_802786
2025-02-27 14:44:20 +0000   56126 execution.bulk     INFO     Process 56163 terminated.
2025-02-27 14:44:20 +0000   56126 execution.bulk     WARNING  Process 56167 had been terminated.
2025-02-27 14:44:20 +0000   56126 execution.bulk     WARNING  Process 56158 had been terminated.
2025-02-27 14:44:14 +0000   55947 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-02-27 14:44:15 +0000   55947 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3}.
2025-02-27 14:44:17 +0000   55947 execution.bulk     INFO     Process name(ForkProcess-2:2)-Process id(56163)-Line number(0) start execution.
2025-02-27 14:44:17 +0000   55947 execution.bulk     INFO     Process name(ForkProcess-2:1)-Process id(56

In [13]:
details = pf.get_details(base_run)
details.head(10)

,inputs.question,inputs.line_number,outputs.output
0,What is capital of France?,0,The capital of France is Paris.
1,What is the meaning of life?,1,The meaning of life is a philosophical questio...
2,What are the planets in Sun system?,2,The planets in the Solar System are:\n\n1. Mer...


## 3. Evaluate your flow
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which usually using LLM assert the produced output matches certain expectation. 

### Run evaluation on the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.

In [14]:
eval_prompty = "../eval-basic/eval.prompty"

eval_run = pf.run(
    flow=eval_prompty,
    data="./data.jsonl",  # path to the data file
    run=base_run,  # specify base_run as the run you want to evaluate
    column_mapping={
        "question": "${data.question}",
        "answer": "${run.outputs.output}",  # TODO refine this mapping
        "ground_truth": "${data.ground_truth}",
    },
    stream=True,
)

[2025-02-27 14:44:21 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run eval_basic_20250227_144421_332312, log path: /home/azureuser/.promptflow/.runs/eval_basic_20250227_144421_332312/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=eval_basic_20250227_144421_332312
2025-02-27 14:44:24 +0000   56342 execution.bulk     INFO     Process 56390 terminated.
2025-02-27 14:44:24 +0000   56342 execution.bulk     WARNING  Process 56382 had been terminated.
2025-02-27 14:44:24 +0000   56342 execution.bulk     WARNING  Process 56397 had been terminated.
2025-02-27 14:44:21 +0000   55947 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-02-27 14:44:21 +0000   55947 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3}.
2025-02-27 14:44:23 +0000   55947 execution.bulk     INFO     Process name(ForkProcess-4:2)-Process id(56390)-Line number(0) start execution.
2025-02-27 14:44:23 +0000   55947 execution.bulk     INFO     Process name(ForkProcess-4:3)-Process 

In [15]:
details = pf.get_details(eval_run)
details.head(10)

,inputs.question,inputs.answer,inputs.ground_truth,inputs.line_number,outputs.score,outputs.explanation
0,What is capital of France?,The capital of France is Paris.,Paris,0,5,The answer correctly identifies Paris as the c...
1,What is the meaning of life?,The meaning of life is a philosophical questio...,The meaning of life is subjective and can vary...,1,5,The answer provides a comprehensive explanatio...
2,What are the planets in Sun system?,The planets in the Solar System are:\n\n1. Mer...,"The planets in the Solar System are Mercury, V...",2,5,The answer correctly lists all the planets in ...


In [16]:
# visualize run using ui
pf.visualize([base_run, eval_run])

Prompt flow service has started...
The HTML file is generated at '/tmp/pf-visualize-detail-gr2anlqn.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


## Next steps

By now you've successfully run your first prompt flow and even did evaluation on it. That's great!

You can check out more examples:
- [Basic Chat](https://github.com/microsoft/promptflow/tree/main/examples/prompty/chat-basic): demonstrates how to create a chatbot that can remember previous interactions and use the conversation history to generate next message.